In [1]:
from numpy import *
import operator
from os import listdir
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import *
from scipy.sparse.linalg import svds
from scipy.stats.stats import pearsonr
from numpy import linalg as la

In [21]:
#user_movie_matrix = pd.read_csv("usermovie.csv",index_col='id')
user_movie_matrix = pd.read_csv("usermovie-headerless.csv")
user_movie_matrix = user_movie_matrix.fillna(0)
print(user_movie_matrix)

with open("usermovie-headerless.csv") as f:
    ncols = len(f.readline().split('\t'))

#R = pd.DataFrame(loadtxt('factorization_1_user_mvrating.csv',delimiter='\t', skiprows=1, usecols=range(1,ncols)))
R = loadtxt('usermovie-headerless.csv',delimiter='\t', skiprows=1, usecols=range(1,ncols))
print(R)

     5    4  Unnamed: 2    2  Unnamed: 4  Unnamed: 5
0  0.0  3.0         0.0  4.0         5.0         0.0
1  0.0  1.0         1.0  1.0         1.0         1.0
2  5.0  3.0         0.0  0.0         1.0         2.0
3  0.0  0.0         4.0  4.0         0.0         0.0
4  0.0  3.0         0.0  5.0         0.0         4.0


ValueError: could not convert string to float: ',3,,4,5,'

In [9]:
kvm = user_movie_matrix>0.5
kvm[kvm == True] = 1
kvm[kvm == False] = 0
# To be consistent with our R matrix
kvm = kvm.astype(float64, copy=False)

print(kvm)

     m1   m2   m3   m4   m5   m6
id                              
1   1.0  1.0  0.0  1.0  0.0  0.0
2   0.0  1.0  0.0  1.0  1.0  0.0
3   0.0  1.0  1.0  1.0  1.0  1.0
4   1.0  1.0  0.0  0.0  1.0  1.0
5   0.0  0.0  1.0  1.0  0.0  0.0
6   0.0  1.0  0.0  1.0  0.0  1.0


In [13]:
k_topics = 3
U, s, V = svds(user_movie_matrix, k=k_topics)
Sig= zeros((k_topics, k_topics), dtype=complex)
Sig = diag(s)

'''
print(U)
print("------------")
print(s)
print("------------")
print(V)
print("------------")
print(Sig)
print("------------")
'''

[[-0.11798204  0.47932837 -0.49001145]
 [ 0.7807037  -0.3421447  -0.48745929]
 [ 0.01020258 -0.1150368  -0.16233937]
 [-0.03285434  0.57028591 -0.40475824]
 [-0.49657914 -0.46412793 -0.25190551]
 [-0.35889781 -0.31513028 -0.51829772]]
------------
[ 4.79963953  7.14808652 11.17268508]
------------
[[-0.15713304  0.14691292 -0.41172133 -0.18412889  0.80857462 -0.31066861]
 [ 0.73419248  0.21562546 -0.27581487 -0.55359179 -0.17563783 -0.03287399]
 [-0.40042733 -0.56870223 -0.10471623 -0.59889902 -0.26890529 -0.27254386]]
------------
[[ 4.79963953  0.          0.        ]
 [ 0.          7.14808652  0.        ]
 [ 0.          0.         11.17268508]]
------------


In [15]:
Q= 

#Sig_df = pd.DataFrame(Sig)
Sig_df = Sig

#V_df = pd.DataFrame(V)
V_df = V

P = dot(Sig_df,V_df)

#lb = regularization constant.
ld=0.1
#reg_value = ld*(sum(Q**2) + sum(P**2))

weighted_errors = []
n_iterations = 10


print(Q)
print("------------")
print(P)
print("------------")

[[-0.11798204  0.47932837 -0.49001145]
 [ 0.7807037  -0.3421447  -0.48745929]
 [ 0.01020258 -0.1150368  -0.16233937]
 [-0.03285434  0.57028591 -0.40475824]
 [-0.49657914 -0.46412793 -0.25190551]
 [-0.35889781 -0.31513028 -0.51829772]]
------------
[[-0.75418193  0.70512906 -1.97611396 -0.88375228  3.88086673 -1.49109733]
 [ 5.24807139  1.54130945 -1.97154854 -3.95712201 -1.2554744  -0.23498612]
 [-4.47384844 -6.35393092 -1.16996142 -6.6913101  -3.00439408 -3.04504673]]
------------


In [19]:
R = user_movie_matrix

print(kvm)

for ii in range(n_iterations):
    for u, Wu in enumerate(kvm):
        print(u,Wu)
        print(diag(Wu))
        Q[u] = linalg.solve(dot(P, dot(diag(Wu), P.T)) + ld * eye(k_topics),dot(P, dot(diag(Wu), R[u].T))).T
        #Q[u] = linalg.solve(dot(P,P.T) + ld * eye(k_topics), dot(P,R[u].T)).T

    for i, Wi in enumerate(kvm.T):
        P[:, i] = linalg.solve(dot(Q.T, dot(diag(Wi), Q)) + ld * eye(k_topics), dot(Q.T, dot(diag(Wi), R[:, i])))
        #P[:, i] = linalg.solve(dot(Q.T, Q) + ld * eye(k_topics), dot(Q.T, R[:, i]))


    R_prime = dot(Q, P)

    MSE = sum((kvm * (R - R_prime)) ** 2)
    print("Error = ", MSE)
    weighted_errors.append(MSE)
    #weighted_errors.append(sum((kvm * (R - R_prime))**2))
    #print weighted_errors
    print('{}th iteration is completed'.format(ii))

     m1   m2   m3   m4   m5   m6
id                              
1   1.0  1.0  0.0  1.0  0.0  0.0
2   0.0  1.0  0.0  1.0  1.0  0.0
3   0.0  1.0  1.0  1.0  1.0  1.0
4   1.0  1.0  0.0  0.0  1.0  1.0
5   0.0  0.0  1.0  1.0  0.0  0.0
6   0.0  1.0  0.0  1.0  0.0  1.0
0 m1


ValueError: Input must be 1- or 2-d.